# Categorical cleanup

Within the feature set, many of the categorical fields contain a large number of unique values. For example, “roof description” alone has over 500 unique values. In order to reduce this, the following steps are applied:

* The unit for average thermal transmittance is standardised to ‘W/m²K’
* The value of average thermal transmittance is reduced to 1 decimal place
* special characters and unnecessary spaces are removed
* spellings are corrected; for example  
    *‘eneregy’ to ‘energy’
* the word ‘assumed’ is removed; for example  
    *‘Solid no insulation (assumed)’ to ‘Solid no insulation’

In [43]:
import numpy as np
import pandas as pd
import datetime
import os
import glob
import json
import re

In [44]:
# set variables from config file
config_path = os.path.abspath('..')

with open(config_path + '/config-example copy.json', 'r') as f:
    config = json.load(f)

processing_path = config['DEFAULT']['processing_path']
epc_train_fname = config['DEFAULT']['epc_train_fname']
epc_test_fname = config['DEFAULT']['epc_test_fname']
epc_train_clean_fname = config['DEFAULT']['epc_train_clean_fname']
epc_test_clean_fname = config['DEFAULT']['epc_test_clean_fname']
epc_fname_suffix = config['DEFAULT']['epc_fname_suffix']

In [45]:
dtype_dict = {'INSPECTION_DATE':'str'}

epc_train = pd.read_csv(os.path.join(processing_path,epc_train_fname) + epc_fname_suffix,
                        header = 0,
                        delimiter = ',',
                        dtype = dtype_dict,
                        parse_dates = ['INSPECTION_DATE'])

epc_test = pd.read_csv(os.path.join(processing_path,epc_test_fname) + epc_fname_suffix,
                        header = 0,
                        delimiter = ',',
                        dtype = dtype_dict,
                        parse_dates = ['INSPECTION_DATE'])

/var/folders/ld/qgzpl61x5_91yljdqrrpz9g00000gn/T/ipykernel_86761/4240386557.py:3: DtypeWarning: Columns (21) have mixed types. Specify dtype option on import or set low_memory=False.
  epc_train = pd.read_csv(os.path.join(processing_path,epc_train_fname) + epc_fname_suffix,
/var/folders/ld/qgzpl61x5_91yljdqrrpz9g00000gn/T/ipykernel_86761/4240386557.py:9: DtypeWarning: Columns (21) have mixed types. Specify dtype option on import or set low_memory=False.
  epc_test = pd.read_csv(os.path.join(processing_path,epc_test_fname) + epc_fname_suffix,


In [46]:
epc_test['BUILT_FORM'].value_counts()

Semi-Detached           299105
Detached                274011
Mid-Terrace             220892
End-Terrace             131252
Enclosed End-Terrace     14013
Enclosed Mid-Terrace      8990
Name: BUILT_FORM, dtype: int64

### Cleaning up units for average thermal transmittance

In [47]:
def thermal_cleanup(df):
    
    ''' 
    standardising the wording in the variables containing average thermal transmittance. Specifically, 
    removing bi-lingual versions, removing =, and standardising units
    Parameters
      df: a dataframe containing variables called 'FLOOR_DESCRIPTION','WALLS_DESCRIPTION','ROOF_DESCRIPTION'
    Returns a dataframe 
    '''
    
    for c in ['FLOOR_DESCRIPTION','WALLS_DESCRIPTION','ROOF_DESCRIPTION']:
        df[c] = df[c].str.replace('  ',' ')
        df[c] = df[c].str.replace('Average thermal transmittance 1 ','Average thermal transmittance 1.00 ')
        df[c] = df[c].str.replace('Average thermal transmittance =','Average thermal transmittance')
        # standardising the unit used
        df[c] = df[c].str.replace(r"W(.*?)K",'W/m²K')
        
    return df

In [48]:
epc_train = thermal_cleanup(epc_train)
epc_test = thermal_cleanup(epc_test)

/var/folders/ld/qgzpl61x5_91yljdqrrpz9g00000gn/T/ipykernel_86761/192030728.py:16: FutureWarning: The default value of regex will change from True to False in a future version.
  df[c] = df[c].str.replace(r"W(.*?)K",'W/m²K')


### Floor description

In [49]:
def thermal_floor(df):
    
    ''' 
    rounds the average thermal transmittance figure which is within a str field to 1 decimal place
    Parameters
      df: a dataframe containing a variable called 'FLOOR_DESCRIPTION'
    Returns a dataframe 
    '''
    
    # finds the decimal number
    df['floors_average_thermal_transmittance'] = df['FLOOR_DESCRIPTION'].str.findall(r'\d.\d*')
    # rounds the number to 1 decimal place
    df['floors_average_thermal_transmittance'] = round(df['floors_average_thermal_transmittance'].str[0].astype(float),1)
    # 
    df['FLOOR_DESCRIPTION'] = df.apply(lambda row: 'average thermal transmittance %.1f W/m²K' % (row['floors_average_thermal_transmittance']) if 'Average' in str(row['FLOOR_DESCRIPTION']) else row['FLOOR_DESCRIPTION'],axis=1)
    
    return df

In [50]:
epc_train = thermal_floor(epc_train)
epc_test = thermal_floor(epc_test)

In [51]:
len(set(epc_train['FLOOR_DESCRIPTION']))

73

In [52]:
def floor_cleanup(df):
    
    ''' 
    cleans-up the FLOOR_DESCRIPTION feature from the EPC dataset
    Parameters
      df: a dataframe containing a variable called 'FLOOR_DESCRIPTION'
    Returns a dataframe 
    '''
    
    df['FLOOR_DESCRIPTION'] = df['FLOOR_DESCRIPTION'].str.lower()

    # removing assumed
    df['FLOOR_DESCRIPTION'] = df['FLOOR_DESCRIPTION'].str.replace("\(assumed\)","")

    # standardising language
    df['FLOOR_DESCRIPTION'] = df['FLOOR_DESCRIPTION'].str.replace("\(another dwelling below\)",'other premises below') 
    df['FLOOR_DESCRIPTION'] = df['FLOOR_DESCRIPTION'].str.replace("\(other premises below\)",'other premises below')
    df['FLOOR_DESCRIPTION'] = df['FLOOR_DESCRIPTION'].str.replace("solid.",'solid,') 
    df['FLOOR_DESCRIPTION'] = df['FLOOR_DESCRIPTION'].str.replace("uninsulated",'no insulation,') 
    df['FLOOR_DESCRIPTION'] = df['FLOOR_DESCRIPTION'].str.replace("insulation=100mm",'100 mm insulation') 
    df['FLOOR_DESCRIPTION'] = df['FLOOR_DESCRIPTION'].str.replace(", \(assumed\)",' (assumed)')
    df['FLOOR_DESCRIPTION'] = df['FLOOR_DESCRIPTION'].str.replace("insulation=25mm",'25 mm insulation')
    df['FLOOR_DESCRIPTION'] = df['FLOOR_DESCRIPTION'].str.replace("insulation=75mm",'75 mm insulation')
    df['FLOOR_DESCRIPTION'] = df['FLOOR_DESCRIPTION'].str.replace("limited insulated",'limited insulation')
    # cleanup
    df['FLOOR_DESCRIPTION'] = df['FLOOR_DESCRIPTION'].str.replace("[ \t]+$","")
    df['FLOOR_DESCRIPTION'] = df['FLOOR_DESCRIPTION'].str.replace(r'\,$','')
    df['FLOOR_DESCRIPTION'] = df['FLOOR_DESCRIPTION'].str.replace(r'^, ','')
    df['FLOOR_DESCRIPTION'] = df['FLOOR_DESCRIPTION'].str.replace(r'\?','')
    
    return df

In [53]:
epc_train = floor_cleanup(epc_train)
epc_test = floor_cleanup(epc_test)

/var/folders/ld/qgzpl61x5_91yljdqrrpz9g00000gn/T/ipykernel_86761/4016219099.py:13: FutureWarning: The default value of regex will change from True to False in a future version.
  df['FLOOR_DESCRIPTION'] = df['FLOOR_DESCRIPTION'].str.replace("\(assumed\)","")
/var/folders/ld/qgzpl61x5_91yljdqrrpz9g00000gn/T/ipykernel_86761/4016219099.py:16: FutureWarning: The default value of regex will change from True to False in a future version.
  df['FLOOR_DESCRIPTION'] = df['FLOOR_DESCRIPTION'].str.replace("\(another dwelling below\)",'other premises below')
/var/folders/ld/qgzpl61x5_91yljdqrrpz9g00000gn/T/ipykernel_86761/4016219099.py:17: FutureWarning: The default value of regex will change from True to False in a future version.
  df['FLOOR_DESCRIPTION'] = df['FLOOR_DESCRIPTION'].str.replace("\(other premises below\)",'other premises below')
/var/folders/ld/qgzpl61x5_91yljdqrrpz9g00000gn/T/ipykernel_86761/4016219099.py:18: FutureWarning: The default value of regex will change from True to False

In [54]:
len(set(epc_train['FLOOR_DESCRIPTION']))

61

### Lighting description

In [55]:
# lighting and second heat descriptions have some entries the wrong way around
epc_train['SECONDHEAT_DESCRIPTION1'] = epc_train.apply(lambda row: row['LIGHTING_DESCRIPTION'] if 'lighting' in str(row['SECONDHEAT_DESCRIPTION']) else row['SECONDHEAT_DESCRIPTION'],axis = 1)
epc_train['LIGHTING_DESCRIPTION1'] = epc_train.apply(lambda row: row['SECONDHEAT_DESCRIPTION'] if 'lighting' in str(row['SECONDHEAT_DESCRIPTION']) else row['LIGHTING_DESCRIPTION'],axis = 1)

epc_test['SECONDHEAT_DESCRIPTION1'] = epc_test.apply(lambda row: row['LIGHTING_DESCRIPTION'] if 'lighting' in str(row['SECONDHEAT_DESCRIPTION']) else row['SECONDHEAT_DESCRIPTION'],axis = 1)
epc_test['LIGHTING_DESCRIPTION1'] = epc_test.apply(lambda row: row['SECONDHEAT_DESCRIPTION'] if 'lighting' in str(row['SECONDHEAT_DESCRIPTION']) else row['LIGHTING_DESCRIPTION'],axis = 1)

In [56]:
epc_train.drop(columns= ['SECONDHEAT_DESCRIPTION','LIGHTING_DESCRIPTION'],axis=1,inplace=True)
epc_test.drop(columns= ['SECONDHEAT_DESCRIPTION','LIGHTING_DESCRIPTION'],axis=1,inplace=True)
epc_train.rename(columns={'SECONDHEAT_DESCRIPTION1':'SECONDHEAT_DESCRIPTION','LIGHTING_DESCRIPTION1':'LIGHTING_DESCRIPTION'},inplace=True)
epc_test.rename(columns={'SECONDHEAT_DESCRIPTION1':'SECONDHEAT_DESCRIPTION','LIGHTING_DESCRIPTION1':'LIGHTING_DESCRIPTION'},inplace=True)

In [57]:
len(set(epc_train['LIGHTING_DESCRIPTION']))

355

In [58]:
def lighting_cleanup(df):
    
    ''' 
    cleans-up the LIGHTING_DESCRIPTION feature from the EPC dataset
    Parameters
      df: a dataframe containing a variable called 'LIGHTING_DESCRIPTION'
    Returns a dataframe 
    '''
    
    df['LIGHTING_DESCRIPTION'] = df['LIGHTING_DESCRIPTION'].str.lower()
    # combining ways of saying all lights are low energy
    df['LIGHTING_DESCRIPTION'] = df['LIGHTING_DESCRIPTION'].str.replace(r'goleuadau ynni-isel ym mhob un o.r mannau gosod','low energy lighting in all fixed outlets')
    df['LIGHTING_DESCRIPTION'] = df['LIGHTING_DESCRIPTION'].str.replace(r'low energy lighting in 120% of fixed outlets','low energy lighting in all fixed outlets')
    df['LIGHTING_DESCRIPTION'] = df['LIGHTING_DESCRIPTION'].str.replace(r'low energy lighting 100% of fixed outlets','low energy lighting in all fixed outlets')
    # translating welsh sentences
    df['LIGHTING_DESCRIPTION'] = df['LIGHTING_DESCRIPTION'].str.replace(r"\|goleuadau(.*)",'') 
    df['LIGHTING_DESCRIPTION'] = df['LIGHTING_DESCRIPTION'].str.replace(r"o.r mannau gosod",'of fixed outlets')
    df['LIGHTING_DESCRIPTION'] = df['LIGHTING_DESCRIPTION'].str.replace('% fixed','% of fixed')
    df['LIGHTING_DESCRIPTION'] = df['LIGHTING_DESCRIPTION'].str.replace('goleuadau ynni-isel mewn','low energy lighting in')
    df['LIGHTING_DESCRIPTION'] = df['LIGHTING_DESCRIPTION'].str.replace(r"\.\d*",'')
    df['LIGHTING_DESCRIPTION'] = df['LIGHTING_DESCRIPTION'].str.replace("dim goleuadau ynni-isel",'no low energy lighting')
    # clean up
    df['LIGHTING_DESCRIPTION'] = df['LIGHTING_DESCRIPTION'].str.replace("eneregy",'energy')
    
    return df

In [59]:
epc_train = lighting_cleanup(epc_train)
epc_test = lighting_cleanup(epc_test)

/var/folders/ld/qgzpl61x5_91yljdqrrpz9g00000gn/T/ipykernel_86761/3194070595.py:12: FutureWarning: The default value of regex will change from True to False in a future version.
  df['LIGHTING_DESCRIPTION'] = df['LIGHTING_DESCRIPTION'].str.replace(r'goleuadau ynni-isel ym mhob un o.r mannau gosod','low energy lighting in all fixed outlets')
/var/folders/ld/qgzpl61x5_91yljdqrrpz9g00000gn/T/ipykernel_86761/3194070595.py:16: FutureWarning: The default value of regex will change from True to False in a future version.
  df['LIGHTING_DESCRIPTION'] = df['LIGHTING_DESCRIPTION'].str.replace(r"\|goleuadau(.*)",'')
/var/folders/ld/qgzpl61x5_91yljdqrrpz9g00000gn/T/ipykernel_86761/3194070595.py:17: FutureWarning: The default value of regex will change from True to False in a future version.
  df['LIGHTING_DESCRIPTION'] = df['LIGHTING_DESCRIPTION'].str.replace(r"o.r mannau gosod",'of fixed outlets')
/var/folders/ld/qgzpl61x5_91yljdqrrpz9g00000gn/T/ipykernel_86761/3194070595.py:20: FutureWarning: The

In [60]:
len(set(epc_train['LIGHTING_DESCRIPTION']))

109

In [61]:
def lighting_perc_cleanup(df):
    
    ''' 
    rounds the low energy lighting percentage figure which is within a str field
    Parameters
      df: a dataframe containing a variable called 'LIGHTING_DESCRIPTION'
    Returns a dataframe 
    '''
    
    df['low_energy_lighting_perc'] = df['LIGHTING_DESCRIPTION'].str.findall(r'(\d*)\%')
    df['low_energy_lighting_perc'] = round(df['low_energy_lighting_perc'].str[0].astype(float),-1)
    df['LIGHTING_DESCRIPTION'] = df.apply(lambda row: 'low energy lighting %d%% of fixed outlets' % (int(row['low_energy_lighting_perc'])) if '%' in str(row['LIGHTING_DESCRIPTION']) else row['LIGHTING_DESCRIPTION'],axis=1)

    return df

In [62]:
epc_train = lighting_perc_cleanup(epc_train)
epc_test = lighting_perc_cleanup(epc_test)

In [63]:
len(set(epc_train['LIGHTING_DESCRIPTION']))

15

### Roof Description

In [64]:
len(set(epc_train['ROOF_DESCRIPTION']))

329

In [65]:
def roof_cleanup(df):
    
    ''' 
    cleans-up the ROOF_DESCRIPTION feature from the EPC dataset
    Parameters
      df: a dataframe containing a variable called 'ROOF_DESCRIPTION'
    Returns a dataframe 
    '''
    
    # standardising units
    df['ROOF_DESCRIPTION'] = df['ROOF_DESCRIPTION'].str.replace(" mm",'mm')
    df['ROOF_DESCRIPTION'] = df['ROOF_DESCRIPTION'].str.replace(">= 300mm",">=300mm")
    df['ROOF_DESCRIPTION'] = df['ROOF_DESCRIPTION'].str.replace(">=300mm","300+mm")
    # translating welsh sentences
    df['ROOF_DESCRIPTION'] = df['ROOF_DESCRIPTION'].str.replace("\|\(eiddo arall uwchben\)","")
    df['ROOF_DESCRIPTION'] = df['ROOF_DESCRIPTION'].str.replace("Ar oleddf","Pitched")
    df['ROOF_DESCRIPTION'] = df['ROOF_DESCRIPTION'].str.replace("dim inswleiddio","no insulation")
    df['ROOF_DESCRIPTION'] = df['ROOF_DESCRIPTION'].str.replace(r"wedi(.*?)i inswleiddio","insulated")
    df['ROOF_DESCRIPTION'] = df['ROOF_DESCRIPTION'].str.replace("(rhagdybiaeth)","assumed")
    df['ROOF_DESCRIPTION'] = df['ROOF_DESCRIPTION'].str.replace(r"lo inswleiddio yn y llof.*","loft insulation")
    df['ROOF_DESCRIPTION'] = df['ROOF_DESCRIPTION'].str.replace(r"o inswleiddio yn y llof.*","loft insulation")
    df['ROOF_DESCRIPTION'] = df['ROOF_DESCRIPTION'].str.replace("Ystafell\(oedd\) to","Roof room(s)")
    df['ROOF_DESCRIPTION'] = df['ROOF_DESCRIPTION'].str.replace(r"wedi(.*?)i hinswleiddio","insulated")
    df['ROOF_DESCRIPTION'] = df['ROOF_DESCRIPTION'].str.replace("nenfwd","ceiling")
    df['ROOF_DESCRIPTION'] = df['ROOF_DESCRIPTION'].str.replace("wrth y trawstia(.*?)","at rafters")
    df['ROOF_DESCRIPTION'] = df['ROOF_DESCRIPTION'].str.replace("inswleiddio cyfyngedig","limited insulation")
    df['ROOF_DESCRIPTION'] = df['ROOF_DESCRIPTION'].str.replace("To gwellt, gydag inswleiddio ychwanegol","Thatched, with additional insulation")
    df['ROOF_DESCRIPTION'] = df['ROOF_DESCRIPTION'].str.replace("Yn wastad","Always")
    # standardising descriptions
    df['ROOF_DESCRIPTION'] = df['ROOF_DESCRIPTION'].str.replace("Roof room,","Roof room(s),")
    df['ROOF_DESCRIPTION'] = df['ROOF_DESCRIPTION'].str.replace("annedd arall uwchben","other premises above")
    df['ROOF_DESCRIPTION'] = df['ROOF_DESCRIPTION'].str.replace("another dwelling above","other premises above")
    df['ROOF_DESCRIPTION'] = df['ROOF_DESCRIPTION'].str.replace("Dwelling Above","(other premises above)")
    df['ROOF_DESCRIPTION'] = df['ROOF_DESCRIPTION'].str.replace(r"0 W/m²K",r" W/m²K")
    df['ROOF_DESCRIPTION'] = df['ROOF_DESCRIPTION'].str.replace("  W/m²K"," 0.0 W/m²K")
    df['ROOF_DESCRIPTION'] = df['ROOF_DESCRIPTION'].str.replace("Roof room\(s\), no insulation\(assumed\)","Roof room(s), no insulation (assumed)")
    df['ROOF_DESCRIPTION'] = df['ROOF_DESCRIPTION'].str.replace("Other premises above","(other premises above)")
    df['ROOF_DESCRIPTION'] = df['ROOF_DESCRIPTION'].str.replace("\(assumed\)","")
    # cleanup
    df['ROOF_DESCRIPTION'] = df['ROOF_DESCRIPTION'].str.replace(r'\.$','')
    df['ROOF_DESCRIPTION'] = df['ROOF_DESCRIPTION'].str.replace(r'\,$','')
    df['ROOF_DESCRIPTION'] = df['ROOF_DESCRIPTION'].str.replace("  \+"," +")
    df['ROOF_DESCRIPTION'] = df['ROOF_DESCRIPTION'].str.replace("[ \t]+$","")
    df['ROOF_DESCRIPTION'] = df['ROOF_DESCRIPTION'].str.replace("mmmm","mm")
    df['ROOF_DESCRIPTION'] = df['ROOF_DESCRIPTION'].str.replace("Thatchedinsulated","Thatched, insulated")
    df['ROOF_DESCRIPTION'] = df['ROOF_DESCRIPTION'].str.replace("\*\*\* INVALID INPUT Code \: 57 \*\*\*","")
    
    return df

In [66]:
epc_train = roof_cleanup(epc_train)
epc_test = roof_cleanup(epc_test)

/var/folders/ld/qgzpl61x5_91yljdqrrpz9g00000gn/T/ipykernel_86761/2604228721.py:15: FutureWarning: The default value of regex will change from True to False in a future version.
  df['ROOF_DESCRIPTION'] = df['ROOF_DESCRIPTION'].str.replace("\|\(eiddo arall uwchben\)","")
/var/folders/ld/qgzpl61x5_91yljdqrrpz9g00000gn/T/ipykernel_86761/2604228721.py:18: FutureWarning: The default value of regex will change from True to False in a future version.
  df['ROOF_DESCRIPTION'] = df['ROOF_DESCRIPTION'].str.replace(r"wedi(.*?)i inswleiddio","insulated")
/var/folders/ld/qgzpl61x5_91yljdqrrpz9g00000gn/T/ipykernel_86761/2604228721.py:19: FutureWarning: The default value of regex will change from True to False in a future version.
  df['ROOF_DESCRIPTION'] = df['ROOF_DESCRIPTION'].str.replace("(rhagdybiaeth)","assumed")
/var/folders/ld/qgzpl61x5_91yljdqrrpz9g00000gn/T/ipykernel_86761/2604228721.py:20: FutureWarning: The default value of regex will change from True to False in a future version.
  df['R

In [67]:
len(set(epc_train['ROOF_DESCRIPTION']))

267

In [68]:
def thermal_roof(df):
    
    ''' 
    rounds the average thermal transmittance figure which is within a str field
    Parameters
      df: a dataframe containing a variable called 'ROOF_DESCRIPTION'
    Returns a dataframe 
    '''
    
    df['roof_average_thermal_transmittance'] = df['ROOF_DESCRIPTION'].str.findall(r'(\d.\d*) W/m²K')
    df['roof_average_thermal_transmittance'] = round(df['roof_average_thermal_transmittance'].str[0].astype(float),1)
    df['ROOF_DESCRIPTION'] = df.apply(lambda row: 'average thermal transmittance %.1f w/m²k' % (row['roof_average_thermal_transmittance']) if 'Average' in str(row['ROOF_DESCRIPTION']) else row['ROOF_DESCRIPTION'],axis=1)
    
    return df

In [69]:
epc_train = thermal_roof(epc_train)
epc_test = thermal_roof(epc_test)

In [70]:
len(set(epc_test['ROOF_DESCRIPTION']))

73

### Walls description

In [71]:
len(set(epc_train['WALLS_DESCRIPTION']))

392

In [72]:
def walls_cleanup(df): 
    
    ''' 
    cleans-up the WALLS_DESCRIPTION feature from the EPC dataset
    Parameters
      df: a dataframe containing a variable called 'WALLS_DESCRIPTION'
    Returns a dataframe 
    '''
    
    # removing phrases like as built which aren't adding anything
    df['WALLS_DESCRIPTION'] = df['WALLS_DESCRIPTION'].str.lower()
    df['WALLS_DESCRIPTION'] = df['WALLS_DESCRIPTION'].str.replace(r"fel y(.*?)u hadeiladwyd, ","")
    df['WALLS_DESCRIPTION'] = df['WALLS_DESCRIPTION'].str.replace("as built, ","")
    df['WALLS_DESCRIPTION'] = df['WALLS_DESCRIPTION'].str.replace("cavity\.","cavity wall,")
    df['WALLS_DESCRIPTION'] = df['WALLS_DESCRIPTION'].str.replace("\(rhagdybiaeth\)","(assumed)")
    df['WALLS_DESCRIPTION'] = df['WALLS_DESCRIPTION'].str.replace("\(assumed\)","")
    # translating welsh sentences
    df['WALLS_DESCRIPTION'] = df['WALLS_DESCRIPTION'].str.replace("waliau ceudod","cavity wall")
    df['WALLS_DESCRIPTION'] = df['WALLS_DESCRIPTION'].str.replace("dim inswleiddio","no insulation")
    df['WALLS_DESCRIPTION'] = df['WALLS_DESCRIPTION'].str.replace("tywodfaen","sandstone")
    df['WALLS_DESCRIPTION'] = df['WALLS_DESCRIPTION'].str.replace("ceudod wedi(.*?)i lenwi","filled cavity")
    df['WALLS_DESCRIPTION'] = df['WALLS_DESCRIPTION'].str.replace(r"wedi(.*?)u hinswleiddio","insulated")
    df['WALLS_DESCRIPTION'] = df['WALLS_DESCRIPTION'].str.replace(r"ffr(.*?)m bren","timber frame")
    df['WALLS_DESCRIPTION'] = df['WALLS_DESCRIPTION'].str.replace("briciau solet","solid brick")
    df['WALLS_DESCRIPTION'] = df['WALLS_DESCRIPTION'].str.replace("wedi(.*?)u hadeiladu yn (.*?)l system","system built")
    df['WALLS_DESCRIPTION'] = df['WALLS_DESCRIPTION'].str.replace("inswleiddio rhannol","partial insulation")
    df['WALLS_DESCRIPTION'] = df['WALLS_DESCRIPTION'].str.replace("gydag inswleiddio allanol","with external insulation")
    df['WALLS_DESCRIPTION'] = df['WALLS_DESCRIPTION'].str.replace("gwenithfaen neu risgraig","granite or whinstone")
    df['WALLS_DESCRIPTION'] = df['WALLS_DESCRIPTION'].str.replace("gydag inswleiddio mewnol","with internal insulation")
    # standardising punctuation
    df['WALLS_DESCRIPTION'] = df['WALLS_DESCRIPTION'].str.replace("solid brick\.","solid brick,")
    # standardising language
    df['WALLS_DESCRIPTION'] = df['WALLS_DESCRIPTION'].str.replace("granite or whin,","granite or whinstone,")
    df['WALLS_DESCRIPTION'] = df['WALLS_DESCRIPTION'].str.replace("stone \(granite or whin\)\.","granite or whinstone,")
    df['WALLS_DESCRIPTION'] = df['WALLS_DESCRIPTION'].str.replace("with external insulation","insulated")
    df['WALLS_DESCRIPTION'] = df['WALLS_DESCRIPTION'].str.replace("with internal insulation","insulated")
    df['WALLS_DESCRIPTION'] = df['WALLS_DESCRIPTION'].str.replace("with additional insulation","insulated")
    df['WALLS_DESCRIPTION'] = df['WALLS_DESCRIPTION'].str.replace("with insulation","insulated")
    # clean up
    df['WALLS_DESCRIPTION'] = df['WALLS_DESCRIPTION'].str.replace(r'\.$','')
    df['WALLS_DESCRIPTION'] = df['WALLS_DESCRIPTION'].str.replace('\+ chr\(13\) \+','+')
    df['WALLS_DESCRIPTION'] = df['WALLS_DESCRIPTION'].str.replace("  \+"," +")
    df['WALLS_DESCRIPTION'] = df['WALLS_DESCRIPTION'].str.replace("[ \t]+$","")
    df['WALLS_DESCRIPTION'] = df['WALLS_DESCRIPTION'].str.replace('timber frame\.','timber frame,')
    df['WALLS_DESCRIPTION'] = df['WALLS_DESCRIPTION'].str.replace('\?','')
    df['WALLS_DESCRIPTION'] = df['WALLS_DESCRIPTION'].str.replace('system built\.','system built,')
    
    return df

In [73]:
epc_train = walls_cleanup(epc_train)
epc_test = walls_cleanup(epc_test)

/var/folders/ld/qgzpl61x5_91yljdqrrpz9g00000gn/T/ipykernel_86761/194093926.py:12: FutureWarning: The default value of regex will change from True to False in a future version.
  df['WALLS_DESCRIPTION'] = df['WALLS_DESCRIPTION'].str.replace(r"fel y(.*?)u hadeiladwyd, ","")
/var/folders/ld/qgzpl61x5_91yljdqrrpz9g00000gn/T/ipykernel_86761/194093926.py:14: FutureWarning: The default value of regex will change from True to False in a future version.
  df['WALLS_DESCRIPTION'] = df['WALLS_DESCRIPTION'].str.replace("cavity\.","cavity wall,")
/var/folders/ld/qgzpl61x5_91yljdqrrpz9g00000gn/T/ipykernel_86761/194093926.py:15: FutureWarning: The default value of regex will change from True to False in a future version.
  df['WALLS_DESCRIPTION'] = df['WALLS_DESCRIPTION'].str.replace("\(rhagdybiaeth\)","(assumed)")
/var/folders/ld/qgzpl61x5_91yljdqrrpz9g00000gn/T/ipykernel_86761/194093926.py:16: FutureWarning: The default value of regex will change from True to False in a future version.
  df['WALLS_

In [74]:
len(set(epc_train['WALLS_DESCRIPTION']))

352

In [75]:
def thermal_walls(df):
    
    ''' 
    rounds the average thermal transmittance figure which is within a str field
    Parameters
      df: a dataframe containing a variable called 'WALLS_DESCRIPTION'
    Returns a dataframe 
    '''
    
    df['walls_average_thermal_transmittance'] = df['WALLS_DESCRIPTION'].str.findall(r'\d.\d\d')
    df['walls_average_thermal_transmittance'] = round(df['walls_average_thermal_transmittance'].str[0].astype(float),1)
    df['WALLS_DESCRIPTION'] = df.apply(lambda row: 'average thermal transmittance %.1f w/m²k' % (row['walls_average_thermal_transmittance']) if 'average' in str(row['WALLS_DESCRIPTION']) else row['WALLS_DESCRIPTION'],axis=1)
    
    return df

In [76]:
epc_train = thermal_walls(epc_train)
epc_test = thermal_walls(epc_test)

In [77]:
len(set(epc_train['WALLS_DESCRIPTION']))

77

### windows description

In [78]:
len(set(epc_train['WINDOWS_DESCRIPTION']))

56

In [79]:
def windows_cleanup(df): 
    
    ''' 
    cleans-up the WINDOWS_DESCRIPTION feature from the EPC dataset
    Parameters
      df: a dataframe containing a variable called 'WINDOWS_DESCRIPTION'
    Returns a dataframe 
    '''
    
    df['WINDOWS_DESCRIPTION'] = df['WINDOWS_DESCRIPTION'].str.lower()
    # translating welsh sentences
    df['WINDOWS_DESCRIPTION'] = df['WINDOWS_DESCRIPTION'].str.replace("ffenestri perfformiad uchel","high performance glazing")
    df['WINDOWS_DESCRIPTION'] = df['WINDOWS_DESCRIPTION'].str.replace("gwydrau dwbl gan mwyaf","mostly double glazing")
    df['WINDOWS_DESCRIPTION'] = df['WINDOWS_DESCRIPTION'].str.replace("rhai gwydrau dwbl","partial double glazing")
    df['WINDOWS_DESCRIPTION'] = df['WINDOWS_DESCRIPTION'].str.replace("gwydrau sengl","single glazing")
    df['WINDOWS_DESCRIPTION'] = df['WINDOWS_DESCRIPTION'].str.replace("gwydrau dwbl rhannol","partial double glazing")
    df['WINDOWS_DESCRIPTION'] = df['WINDOWS_DESCRIPTION'].str.replace("gwydrau dwbl llawn","fully double glazing")
    df['WINDOWS_DESCRIPTION'] = df['WINDOWS_DESCRIPTION'].str.replace("gwydrau lluosog ym mhobman","multiple glazing throughout")
    df['WINDOWS_DESCRIPTION'] = df['WINDOWS_DESCRIPTION'].str.replace("gwydrau eilaidd llawn","full secondary glazing")
    # standardising language
    df['WINDOWS_DESCRIPTION'] = df['WINDOWS_DESCRIPTION'].str.replace("glazed","glazing")
    df['WINDOWS_DESCRIPTION'] = df['WINDOWS_DESCRIPTION'].str.replace("fully","full")
    df['WINDOWS_DESCRIPTION'] = df['WINDOWS_DESCRIPTION'].str.replace("fully","full")
    # cleanup
    df['WINDOWS_DESCRIPTION'] = df['WINDOWS_DESCRIPTION'].str.replace("single glazingsingle glazing","single glazing")
    df['WINDOWS_DESCRIPTION'] = df['WINDOWS_DESCRIPTION'].str.replace("single glazingdouble glazing","single glazing and double glazing")
    df['WINDOWS_DESCRIPTION'] = df['WINDOWS_DESCRIPTION'].str.replace("single glazingsecondary glazing","single glazing and secondary glazing")
    df['WINDOWS_DESCRIPTION'] = df['WINDOWS_DESCRIPTION'].str.replace("[ \t]+$","")
    df['WINDOWS_DESCRIPTION'] = df['WINDOWS_DESCRIPTION'].str.replace("  "," ")
    
    return df

In [80]:
epc_train = windows_cleanup(epc_train)
epc_test = windows_cleanup(epc_test)

/var/folders/ld/qgzpl61x5_91yljdqrrpz9g00000gn/T/ipykernel_86761/1212584499.py:28: FutureWarning: The default value of regex will change from True to False in a future version.
  df['WINDOWS_DESCRIPTION'] = df['WINDOWS_DESCRIPTION'].str.replace("[ \t]+$","")


In [81]:
len(set(epc_train['WINDOWS_DESCRIPTION']))

37

### hotwater description

In [82]:
def hotwater_cleanup(df):
    
    ''' 
    cleans-up the HOTWATER_DESCRIPTION feature from the EPC dataset
    Parameters
      df: a dataframe containing a variable called 'HOTWATER_DESCRIPTION'
    Returns a dataframe 
    '''
    
    # removing spaces at the end
    df['HOTWATER_DESCRIPTION'] = df['HOTWATER_DESCRIPTION'].str.replace("[ \t]+$","")

    # translating welsh sentences
    df['HOTWATER_DESCRIPTION'] = df['HOTWATER_DESCRIPTION'].str.replace(r"O(.*?)r brif system","From main system")
    df['HOTWATER_DESCRIPTION'] = df['HOTWATER_DESCRIPTION'].str.replace(r"Trochi trydan","Electric immersion")
    df['HOTWATER_DESCRIPTION'] = df['HOTWATER_DESCRIPTION'].str.replace(r"an-frig","off-peak")
    df['HOTWATER_DESCRIPTION'] = df['HOTWATER_DESCRIPTION'].str.replace(r"O system eilaidd","From secondary system")
    df['HOTWATER_DESCRIPTION'] = df['HOTWATER_DESCRIPTION'].str.replace(r"Nwy wrth fwy nag un pwynt","Gas multipoint")
    df['HOTWATER_DESCRIPTION'] = df['HOTWATER_DESCRIPTION'].str.replace(r"Popty estynedig olew","Oil range cooker")
    df['HOTWATER_DESCRIPTION'] = df['HOTWATER_DESCRIPTION'].str.replace(r"tarriff safonol","standard tariff")
    df['HOTWATER_DESCRIPTION'] = df['HOTWATER_DESCRIPTION'].str.replace(r"Dim system ar gael","No system present")
    df['HOTWATER_DESCRIPTION'] = df['HOTWATER_DESCRIPTION'].str.replace(r"adfer gwres nwyon ffliw","flue gas heat recovery")
    df['HOTWATER_DESCRIPTION'] = df['HOTWATER_DESCRIPTION'].str.replace(r"gydag ynni(.*?)r haul","plus solar")
    df['HOTWATER_DESCRIPTION'] = df['HOTWATER_DESCRIPTION'].str.replace(r"dim thermostat ar y silindr","no cylinderstat")
    df['HOTWATER_DESCRIPTION'] = df['HOTWATER_DESCRIPTION'].str.replace(r"rhagdybir bod twymwr tanddwr trydan","electric immersion assumed")
    df['HOTWATER_DESCRIPTION'] = df['HOTWATER_DESCRIPTION'].str.replace(r"an-frig","off peak")
    df['HOTWATER_DESCRIPTION'] = df['HOTWATER_DESCRIPTION'].str.replace(r"Twymwr tanddwr","underfloor heating")

    # standardising language
    df['HOTWATER_DESCRIPTION'] = df['HOTWATER_DESCRIPTION'].str.replace("cylinder thermostat","cylinderstat")
    df['HOTWATER_DESCRIPTION'] = df['HOTWATER_DESCRIPTION'].str.replace("No system present :","No system present:")
    df['HOTWATER_DESCRIPTION'] = df['HOTWATER_DESCRIPTION'].str.replace("No hot water system present -","No system present:")
    df['HOTWATER_DESCRIPTION'] = df['HOTWATER_DESCRIPTION'].str.replace("From community scheme","Community scheme")
    df['HOTWATER_DESCRIPTION'] = df['HOTWATER_DESCRIPTION'].str.replace("Community heat pump","Community scheme with CHP")
    df['HOTWATER_DESCRIPTION'] = df['HOTWATER_DESCRIPTION'].str.replace("From secondary heater","From secondary system")
    df['HOTWATER_DESCRIPTION'] = df['HOTWATER_DESCRIPTION'].str.replace("SAP05:Hot-Water","SAP:Hot-Water")
    df['HOTWATER_DESCRIPTION'] = df['HOTWATER_DESCRIPTION'].str.replace("community scheme","Community scheme")
    df['HOTWATER_DESCRIPTION'] = df['HOTWATER_DESCRIPTION'].str.replace("plus solar, no cylinderstat","no cylinderstat, plus solar")
    df['HOTWATER_DESCRIPTION'] = df['HOTWATER_DESCRIPTION'].str.replace("From second main heating system","From secondary system")
    df['HOTWATER_DESCRIPTION'] = df['HOTWATER_DESCRIPTION'].str.replace("none","No system present: electric immersion assumed")
    df['HOTWATER_DESCRIPTION'] = df['HOTWATER_DESCRIPTION'].str.replace("plus solar, flue gas heat recovery","flue gas heat recovery, plus solar")
    df['HOTWATER_DESCRIPTION'] = df['HOTWATER_DESCRIPTION'].str.replace("no cylinderstat, no cylinderstat","no cylinderstat")

    # cleanup
    df['HOTWATER_DESCRIPTION'] = df['HOTWATER_DESCRIPTION'].str.replace("No system present\?electric immersion assumed","No system present: electric immersion assumed")
    df['HOTWATER_DESCRIPTION'] = df['HOTWATER_DESCRIPTION'].replace("***SAMPLE***",np.nan)

    return df

In [83]:
epc_train = hotwater_cleanup(epc_train)
epc_test = hotwater_cleanup(epc_test)

/var/folders/ld/qgzpl61x5_91yljdqrrpz9g00000gn/T/ipykernel_86761/1945241965.py:11: FutureWarning: The default value of regex will change from True to False in a future version.
  df['HOTWATER_DESCRIPTION'] = df['HOTWATER_DESCRIPTION'].str.replace("[ \t]+$","")
/var/folders/ld/qgzpl61x5_91yljdqrrpz9g00000gn/T/ipykernel_86761/1945241965.py:14: FutureWarning: The default value of regex will change from True to False in a future version.
  df['HOTWATER_DESCRIPTION'] = df['HOTWATER_DESCRIPTION'].str.replace(r"O(.*?)r brif system","From main system")
/var/folders/ld/qgzpl61x5_91yljdqrrpz9g00000gn/T/ipykernel_86761/1945241965.py:23: FutureWarning: The default value of regex will change from True to False in a future version.
  df['HOTWATER_DESCRIPTION'] = df['HOTWATER_DESCRIPTION'].str.replace(r"gydag ynni(.*?)r haul","plus solar")
/var/folders/ld/qgzpl61x5_91yljdqrrpz9g00000gn/T/ipykernel_86761/1945241965.py:45: FutureWarning: The default value of regex will change from True to False in a fu

In [84]:
len(set(epc_train['HOTWATER_DESCRIPTION']))

117

### main heating controls

In [85]:
len(set(epc_train['MAIN_HEATING_CONTROLS']))

626920

In [86]:
def heat_control_cleanup(df):
    
    ''' 
    cleans-up the MAIN_HEATING_CONTROLS feature from the EPC dataset
    Parameters
      df: a dataframe containing a variable called 'MAIN_HEATING_CONTROLS'
    Returns a dataframe 
    '''
    
    df['MAIN_HEATING_CONTROLS'] = df['MAIN_HEATING_CONTROLS'].str.lower()

    # translating welsh sentences
    df['MAIN_HEATING_CONTROLS'] = df['MAIN_HEATING_CONTROLS'].str.replace(r"rheoli.r t.l . llaw","manual charge control")
    df['MAIN_HEATING_CONTROLS'] = df['MAIN_HEATING_CONTROLS'].str.replace("rhaglennydd, dim thermostat ystafell","programmer, no room thermostat")
    df['MAIN_HEATING_CONTROLS'] = df['MAIN_HEATING_CONTROLS'].str.replace("rheolaeth amser a rheolaeth parthau tymheredd","time and temperature zone control")
    df['MAIN_HEATING_CONTROLS'] = df['MAIN_HEATING_CONTROLS'].str.replace("rhaglennydd a thermostat ystafell","programmer and room thermostat")
    df['MAIN_HEATING_CONTROLS'] = df['MAIN_HEATING_CONTROLS'].str.replace("rhaglennydd a thermostatau ar y cyfarpar","programmer and appliance thermostats")
    df['MAIN_HEATING_CONTROLS'] = df['MAIN_HEATING_CONTROLS'].str.replace("rhaglennydd ac o leiaf ddau thermostat ystafell","programmer and at least two room thermostats")
    df['MAIN_HEATING_CONTROLS'] = df['MAIN_HEATING_CONTROLS'].str.replace("thermostat ystafell yn unig","room thermostat only")
    df['MAIN_HEATING_CONTROLS'] = df['MAIN_HEATING_CONTROLS'].str.replace("dim rheolaeth thermostatig ar dymheredd yr ystafell","no thermostatic control of room temperature")
    df['MAIN_HEATING_CONTROLS'] = df['MAIN_HEATING_CONTROLS'].str.replace("rheoli gwefr drydanol yn awtomatig","automatic charge control")
    df['MAIN_HEATING_CONTROLS'] = df['MAIN_HEATING_CONTROLS'].str.replace("dim rheolaeth amser na rheolaeth thermostatig ar dymheredd yr ystafell","no time or thermostatic control of room temperature")
    df['MAIN_HEATING_CONTROLS'] = df['MAIN_HEATING_CONTROLS'].str.replace("trvs a falf osgoi","trvs and bypass")
    df['MAIN_HEATING_CONTROLS'] = df['MAIN_HEATING_CONTROLS'].str.replace("rhaglennydd","programmer")
    df['MAIN_HEATING_CONTROLS'] = df['MAIN_HEATING_CONTROLS'].str.replace("tal un gyfradd, thermostat ystafell yn unig","flat rate charging, room thermostat only")
    df['MAIN_HEATING_CONTROLS'] = df['MAIN_HEATING_CONTROLS'].str.replace("thermostat ystafell a trvs","room thermostat and trvs")
    df['MAIN_HEATING_CONTROLS'] = df['MAIN_HEATING_CONTROLS'].str.replace("t(.*?)l un gyfradd","flat rate charging")
    df['MAIN_HEATING_CONTROLS'] = df['MAIN_HEATING_CONTROLS'].str.replace("thermostatau ar y cyfarpar","appliance thermostat")
    df['MAIN_HEATING_CONTROLS'] = df['MAIN_HEATING_CONTROLS'].str.replace("dim","none")

    # standardising language
    df['MAIN_HEATING_CONTROLS'] = df['MAIN_HEATING_CONTROLS'].str.replace("programmer, no thermostat","programmer, no room thermostat")
    df['MAIN_HEATING_CONTROLS'] = df['MAIN_HEATING_CONTROLS'].str.replace("flat rate charging\*","flat rate charging")
    df['MAIN_HEATING_CONTROLS'] = df['MAIN_HEATING_CONTROLS'].str.replace("\+","and")
    df['MAIN_HEATING_CONTROLS'] = df['MAIN_HEATING_CONTROLS'].str.replace("\&","and")
    df['MAIN_HEATING_CONTROLS'] = df['MAIN_HEATING_CONTROLS'].str.replace("trv.s","trvs")
    df['MAIN_HEATING_CONTROLS'] = df['MAIN_HEATING_CONTROLS'].str.replace("thermostats","thermostat")
    df['MAIN_HEATING_CONTROLS'] = df['MAIN_HEATING_CONTROLS'].str.replace("communit ","community ")
    df['MAIN_HEATING_CONTROLS'] = df['MAIN_HEATING_CONTROLS'].str.replace("to the use of community heating","to use of community heating")
    df['MAIN_HEATING_CONTROLS'] = df['MAIN_HEATING_CONTROLS'].str.replace(" stat"," thermostat")
    df['MAIN_HEATING_CONTROLS'] = df['MAIN_HEATING_CONTROLS'].str.replace("controls","control")
    df['MAIN_HEATING_CONTROLS'] = df['MAIN_HEATING_CONTROLS'].str.replace("prog ","programmer ")
    df['MAIN_HEATING_CONTROLS'] = df['MAIN_HEATING_CONTROLS'].str.replace("program ","programmer ")
    df['MAIN_HEATING_CONTROLS'] = df['MAIN_HEATING_CONTROLS'].str.replace("programmerand","programmer and")
    df['MAIN_HEATING_CONTROLS'] = df['MAIN_HEATING_CONTROLS'].str.replace("delayed start thermostat and program and trvs","delayed start thermostat, program and trvs")
    df['MAIN_HEATING_CONTROLS'] = df['MAIN_HEATING_CONTROLS'].str.replace("thermostatic","thermostat")
    df['MAIN_HEATING_CONTROLS'] = df['MAIN_HEATING_CONTROLS'].str.replace("flat rate charging, programmer no room thermostat","flat rate charging, programmer, no room thermostat")
    df['MAIN_HEATING_CONTROLS'] = df['MAIN_HEATING_CONTROLS'].str.replace(" 2 "," two ")
    df['MAIN_HEATING_CONTROLS'] = df['MAIN_HEATING_CONTROLS'].str.replace("roomstat","room thermostat")
    df['MAIN_HEATING_CONTROLS'] = df['MAIN_HEATING_CONTROLS'].str.replace("programmer and room thermostat and trvs","programmer, room thermostat and trvs")
    df['MAIN_HEATING_CONTROLS'] = df['MAIN_HEATING_CONTROLS'].str.replace("programmer and trvs and boiler energy manager","programmer, trvs and boiler energy manager")
    df['MAIN_HEATING_CONTROLS'] = df['MAIN_HEATING_CONTROLS'].str.replace("programmer and trvs and bypass","programmer, trvs and bypass")
    df['MAIN_HEATING_CONTROLS'] = df['MAIN_HEATING_CONTROLS'].str.replace("programmer and trvs and flow switch","programmer, trvs and flow switch")
    df['MAIN_HEATING_CONTROLS'] = df['MAIN_HEATING_CONTROLS'].str.replace("temp+$","temperature")
    df['MAIN_HEATING_CONTROLS'] = df['MAIN_HEATING_CONTROLS'].str.replace("no thermostat control of room temperature","no thermostat control")
    df['MAIN_HEATING_CONTROLS'] = df['MAIN_HEATING_CONTROLS'].str.replace("appliance thermostat and programmer","programmer and appliance thermostat")
    df['MAIN_HEATING_CONTROLS'] = df['MAIN_HEATING_CONTROLS'].str.replace("delayed start thermostat and programmer and trvs","delayed start thermostat, programmer and trvs")
    df['MAIN_HEATING_CONTROLS'] = df['MAIN_HEATING_CONTROLS'].str.replace("no time or thermostat control of temperature","no time or thermostat control of room temperature")
    df['MAIN_HEATING_CONTROLS'] = df['MAIN_HEATING_CONTROLS'].str.replace("programmer\?and","programmer and")

    
    return df

In [87]:
epc_train = heat_control_cleanup(epc_train)
epc_test = heat_control_cleanup(epc_test)

/var/folders/ld/qgzpl61x5_91yljdqrrpz9g00000gn/T/ipykernel_86761/1223281842.py:13: FutureWarning: The default value of regex will change from True to False in a future version.
  df['MAIN_HEATING_CONTROLS'] = df['MAIN_HEATING_CONTROLS'].str.replace(r"rheoli.r t.l . llaw","manual charge control")
/var/folders/ld/qgzpl61x5_91yljdqrrpz9g00000gn/T/ipykernel_86761/1223281842.py:27: FutureWarning: The default value of regex will change from True to False in a future version.
  df['MAIN_HEATING_CONTROLS'] = df['MAIN_HEATING_CONTROLS'].str.replace("t(.*?)l un gyfradd","flat rate charging")
/var/folders/ld/qgzpl61x5_91yljdqrrpz9g00000gn/T/ipykernel_86761/1223281842.py:33: FutureWarning: The default value of regex will change from True to False in a future version.
  df['MAIN_HEATING_CONTROLS'] = df['MAIN_HEATING_CONTROLS'].str.replace("flat rate charging\*","flat rate charging")
/var/folders/ld/qgzpl61x5_91yljdqrrpz9g00000gn/T/ipykernel_86761/1223281842.py:34: FutureWarning: The default value o

In [88]:
len(set(epc_train['MAIN_HEATING_CONTROLS']))

626860

In [89]:
epc_train['TRANSACTION_TYPE'] = epc_train['TRANSACTION_TYPE'].str.replace(" - this is for backwards compatibility only and should not be used","")
epc_test['TRANSACTION_TYPE'] = epc_test['TRANSACTION_TYPE'].str.replace(" - this is for backwards compatibility only and should not be used","")
epc_train['GLAZED_TYPE'] = epc_train['GLAZED_TYPE'].replace('INVALID!',np.nan)
epc_test['GLAZED_TYPE'] = epc_test['GLAZED_TYPE'].replace('INVALID!',np.nan)

### exporting data

In [90]:
epc_train[epc_train.isnull()]

,LMK_KEY,region,POSTCODE,BUILDING_REFERENCE_NUMBER,CURRENT_ENERGY_RATING,CURRENT_ENERGY_EFFICIENCY,PROPERTY_TYPE,BUILT_FORM,INSPECTION_DATE,COUNTY,...,HEAT_LOSS_CORRIDOR,FLOOR_HEIGHT,MECHANICAL_VENTILATION,inspection_year,floors_average_thermal_transmittance,SECONDHEAT_DESCRIPTION,LIGHTING_DESCRIPTION,low_energy_lighting_perc,roof_average_thermal_transmittance,walls_average_thermal_transmittance
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3867184,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3867185,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3867186,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3867187,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [94]:
epc_train.to_csv(os.path.join(processing_path,epc_train_clean_fname) + epc_fname_suffix,index = False)
epc_test.to_csv(os.path.join(processing_path,epc_test_clean_fname) + epc_fname_suffix,index = False)